### 복습 
- ratings_train.txt 데이터 로드 
- document 컬럼의 결측치 제외
- 텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제외, 문자열 좌우 공백 제거)
- document의 중복 데이터를 제거 
- 데이터프레임에서 상위 1000개만 사용
- texts -> 데이터프레임의 document 컬럼의 values
- labels -> 데이터프레임의 label 컬럼의 values
- 토큰화 -> Komoran 
    - 선택하는 품사는 NNP NNG VV VA MAG XR
    - 불용어 하다, 되다, 이다
- 단어 사전 생성 (vocab)
    - 단어들 중 최소 출현 횟수가 2회 
    - 단어 사전에는 <PAD>, <UNK>을 제일 앞에 지정하여 단어사전 생성
- 토큰화 된 문서들을 단어 사전의 위치 값에 맞게 인코딩 (enc_inputs)

In [1]:
import pandas as pd 
import re 
from konlpy.tag import Komoran
from collections import Counter
# import collections

In [20]:
df = pd.read_csv("../data/ratings_train.txt", sep = '\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [21]:
# document 컬럼의 결측치를 제거하시오
df.dropna(subset="document", inplace=True)

In [22]:
def normalize(text):

    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
# document 만 텍스트 정규화 -> Series 데이터에서 normalize() 함수를 실행
# 여러 개의 컬럼의 텍스트를 정규화 -> DataFrame 데이터에서 normalize() 함수를 실행


# Series 데이터에서 각각의 values들을 normalize() 대입 
df['document'].map(normalize)  # 결과를 어디에 대입? -> df['document']
# DataFrame 데이터에서 각각의 values들을 normalize() 대입 
df[ ['id', 'document'] ] = df[ ['id', 'document'] ].astype(str).applymap(normalize)

In [24]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [26]:
# document 컬럼의 중복 데이터를 제거 
df.drop_duplicates(subset='document', inplace=True)

In [28]:
# df = df.head(1000)
df = df[:1000]

In [30]:
# 토큰화 작업 
komoran = Komoran()
allow_pos = ["NNP", "NNG", "VV", "VA", "MAG", "XR"]
stop_word = ['하다', '되다', '이다']

def tokenize(doc):
    tokens = []
    for word, pos in komoran.pos(doc):
        if word not in stop_word and pos in allow_pos:
            tokens.append(word)
    return tokens

In [ ]:
# texts, labels 변수 생성 document, lable 데이터를 대입 
texts, labels = df['document'].values, df['label'].values
# texts들을 이용하여 토큰화 
tokens_list = [ tokenize(doc) for doc in texts ]
tokens_list

In [ ]:
# 최소 등장 횟수 2회인 단어사전을 생성 
# 토큰화 된 tokens_list에서 단어들의 출현 횟수를 확인 
freq = Counter( word for toks in tokens_list for word in toks )